## Exceptions Class Hierarchy
Class `Throwable` is the parent class for all exceptions in Java. Only subclasses of `Throwable` can be stated after `throw` keyword or used as argument inside `catch`.

```
                Throwable
                    |
                +---+---+
                |       |
              Error  Exception
                        |
                        +--- Other Exceptions
                        +--- RuntimeException
```

Errors indicate serious problems that a reasonable application should not try to catch (though we can can catch it). Some of the common `Error` classes are:
- `StackOverflowError`: thrown when there is a very deep recursive call
- `OutOfMemoryError`: thrown when JVM cannot allocate an object due to lack of memory
- `IOError`

Errors are not subject to *catch or specify* requirement (discussed later).

## Checked Vs Unchecked Exception
- **Unchecked**: subclasses of `RuntimeException`. One case where it is common practice to throw a `RuntimeException` is when the user calls a method incorrectly.
- **Checked**: these exceptions are subject to *catch or specify* requirement. This means that a code block that may throw checked exception must be handled inside `try catch` block or `throws` keyword must be specified on the wrapping method to indicate that the calling block should handle the exception.

As an example, the constructor of `FileReader` is defined as `public FileReader(File file) throws FileNotFoundException`. This means any code block that calls the constructor must honor the *catch or specify* requirement:

In [ ]:
// Catch
public void readFile(File f) {
    try {
        Reader reader = new FileReader(f); // this code is inside a try block
                                           // and the exception is caught in the catch block
        // ...
    } catch (FileNotFoundException e) {
    
    } // ...
}

// Or specify
public void readFile2(File file) throws FileNotFoundException {
    Reader reader = new FileReader(file);
    
    // ...
}

To summarise:
- Unchecked exception is for things that the programmer did wrong. Example:
    - Programmer walked off the end of an array
    - Programmer did not check for nulls, etc
- Checked exception is for things that are out of the programmer's control. Or more specifically out of programmer and API designer's control. Example:
    - Disk fills up while writing to the file system
    - File handle limit for the process has been reached and you cannot open any more files
    - Received non JSON input when JSON input was expected

More reading available [here](https://stackoverflow.com/questions/613954/the-case-against-checked-exceptions) and [here](http://literatejava.com/exceptions/checked-exceptions-javas-biggest-mistake/).  

**Guideline:** if a client can reasonably be expected to recover from an exception, make it a checked exception. If a client cannot do anything to recover from the exception, make it an unchecked exception.

### Checked Exceptions and Lambda Expression
Consider the code:

In [ ]:
List<String> files = List.of("plaintext.txt");
char[] buffer = new char[1024];

files.forEach(f -> {
    new FileReader(f).read(buffer);
});

The expression `new FileReader` and `read` throw `FileNotFoundException` and `IOException` respectively. Therefore, it must be caught. Lets wrap the `forEach` expression in a *try-catch* block:

In [ ]:
try {
    files.forEach(f -> {
        new FileReader(f).read(buffer);
    });
} catch (IOException e) {
    e.printStackTrace();
}

The above modification doesn't work (nor does specifying that the enclosing method throws exception). We must catch the exception before it escapes the lambda:

In [ ]:
files.forEach(f -> {
    try {
        new FileReader(f).read(buffer);
    } catch (IOException e) {
        e.printStackTrace();
    }
});

Lambda expressions are supposed to be concise, smaller, and crisp, the above code is not. One way is to use a utility method and rethrow the checked exception as a `RuntimeException`:

In [ ]:
List<String> files = List.of("plaintext.txt");
char[] buffer = new char[1024];

files.stream().map(ThrowingFunction.unchecked(FileReader::new))
        .forEach(ThrowingConsumer.unchecked(fR -> fR.read(buffer)));

@FunctionalInterface
interface ThrowingConsumer<T, E extends Throwable> {
    void consume(T t) throws E;

    static <T, E extends Throwable> Consumer<T> unchecked(ThrowingConsumer<T, E> consumer) {
        return t -> {
            try {
                consumer.consume(t);
            } catch (Throwable e) {
                throw new RuntimeException(e);
            }
        };
    }
}

@FunctionalInterface
interface ThrowingFunction<T, R, E extends Throwable> {
    R apply(T t) throws E;

    static <T, R, E extends Throwable> Function<T, R> unchecked(ThrowingFunction<T, R, E> function) {
        return t -> {
            try {
                return function.apply(t);
            } catch (Throwable e) {
                throw new RuntimeException(e);
            }
        };
    }
}

**Sneaky Throws**

## Try Catch Finally Blocks
### The finally Block
The `finally` block is executed even if there is a `return` or `throw` statement in the `try` or `catch` blocks. After `finally` block control returns to try/catch block where return/throw statements were. Note that the `finally` block may not execute if the JVM exits while the `try` or `catch` code is being executed. 

In [ ]:
class TryCatchFinally {
    public int getSomeNumber() {
        int number = 20;
        
        try {
            System.out.println("In try");
            throw new Exception();
        } catch(Exception e) {
            System.out.println("In catch");
            return number;
        } finally {
            System.out.println("In finally");
            number = 10;
        }
    }
}

// In the above example, the value returned by the catch block is still 20. 
// This is because the return value is put on the stack before the call to finally.

In [ ]:
class TryCatchFinally{
    public int[] getSomeArray(){
        int[] numbers = {1,2,3,4,5};
        
        try{
            System.out.println("In try");
            throw new Exception();
        } catch(Exception e){
            System.out.println("In catch");
            return numbers;
        } finally{
            System.out.println("In finally");
            numbers[0] = 0;
        }
    }
}

// In this example, the array returned is {0,2,3,4,5}

In [ ]:
class TryCatchFinally{
    public int getSomeNumber(){
        int number = 20;
        
        try{
            System.out.println("In try");
            throw new Exception();
        } catch(Exception e){
            System.out.println("In catch");
            return number;
        } finally{
            System.out.println("In finally");
            number = 5;
            return number;
        }
    }
}

// In the above case, the number returned is 5.
// Because the return value that is put on the stack in catch is updated by finally?

In [ ]:
class TryCatchFinally{
    public int number = 20;
    
    public int getSomeNumber(){
        try{
            System.out.println("In try");
            throw new Exception();
        } catch(Exception e){
            System.out.println("In catch");
            return number;
        } finally{
            System.out.println("In finally");
            number = 5;
        }
    }
    
    public static void main(String[] args) {
        TryCatchFinally t = new TryCatchFinally();
        System.out.println(t.getSomeNumber());  // Prints 20
        System.out.println(t.number);  // Prints 5
    }
}

### Catching Multiple Exceptions

In [ ]:
try{

} catch(RuntimeException e){
    e.printStackTrace();
} 
/* catch(IOException e){  --> error to catch a checked exception not thrown anywhere
    
} */
catch(Exception e){        // catching parent exception before child exception is error
    e.printStackTrace();
}

Catching multiple exceptions the Java 1.7+ way:

In [ ]:
try{
    throw new IOException();
} catch(IOException | IndexOutOfBoundsException e){
    System.out.println("Exception caught");
} catch(Exception e){
    System.out.println("Exception caught");
}

We cannot specify both child and parent exceptions inside same catch.

### Try With Resources
Available after 1.7+. Only classes which implement `AutoCloseable` interface can be used as argument inside try(). `Closeable` is the older interface. `AutoCloseable` is the new one which is now `Closeable` interface's parent class. The close method is called as soon as control leaves `try` block which means it runs before `catch` or `finally`.

In [ ]:
try(InputStream in = 
        new BufferedInputStream( 
            new FileInputStream( 
                new File("log.txt")))) {
    // ...
} catch (FileNotFoundException e) {
    e.printStackTrace();    // in is not accessible here
} catch (IOException e) {
    e.printStackTrace();    // in is not accessible here either
}

To fix the problem of `in` variable not accessible inside the catch block, we can try the following:

In [ ]:
InputStream in = null;
try(in =        // this is an error, Java assumes variable in to be a type
        new BufferedInputStream( 
            new FileInputStream( 
                new File("log.txt")))) {
    // ...
} catch (FileNotFoundException e) {
    e.printStackTrace();
} catch (IOException e) {
    e.printStackTrace();    
}

We can also specify multiple resources:

In [ ]:
try ( 
    ZipFile zf = new ZipFile(zipFileName);
    BufferedWriter writer = Files.newBufferedWriter(outputFilePath, charset)
) {

Java 9 introduced improvement to the *try-with-resources* clause:

In [ ]:
InputStream in = 
        new BufferedInputStream( 
            new FileInputStream( 
                new File("log.txt")));
try(in) {
    // ...
} catch (FileNotFoundException e) {
    e.printStackTrace();    // in is accessible here
} catch (IOException e) {
    e.printStackTrace();    // in is accessible here either
}

If multiple resources are opened, they will be closed in reverse order of declaration. There is a difference between normal *try-catch-finally* and *try-with-resources* in terms of which exception gets suppressed.

### Suppressed Exception
Consider the code block below:

In [ ]:
public static void finallySuppressesTry() throws Exception {
    try {
        throw new Exception("Exception thrown from try block");
    } finally {
        throw new Exception("Exception thrown from finally block");
    }
}

public static void main(String[] args) {
    try {
        finallySuppressesTry();
    } catch (Exception e) {
        System.out.println(e.getMessage()); // Prints finally block exception message
                                            // Original exception got lost
    }
}

/* Error log would look something like (if not caught in main):
Exception in thread "main" java.lang.Exception: Exception thrown from finally block
    at exceptionHandling.SuppressedExceptionDemo.finallySuppressesTry(SuppressedExceptionDemo.java:8)
    at exceptionHandling.SuppressedExceptionDemo.main(SuppressedExceptionDemo.java:15)
*/

`Throwable` class has facility to add any suppressed exception to an exception:

In [ ]:
public static void addSuppressedException() throws Exception {
    Throwable t = null;
    try {
        throw new Exception("Exception thrown from try block");
    } catch (Exception e) {
       t = e;
    } finally {
        Exception thrown = new Exception("Exception thrown from finally block");
        thrown.addSuppressed(t);
        throw thrown;
    }
}

public static void main(String[] args) throws Exception {
    try {
        finallySuppressesTry();
    } catch (Exception e) {
        e.getSuppressed(); // to access suppressed exception
    }
}

/* Error log would look something like (if not caught in main):
Exception in thread "main" java.lang.Exception: Exception thrown from finally block
    at exceptionHandling.SuppressedExceptionDemo.addSuppressedException(SuppressedExceptionDemo.java:19)
    at exceptionHandling.SuppressedExceptionDemo.main(SuppressedExceptionDemo.java:37)
    Suppressed: java.lang.Exception: Exception thrown from try block
        at exceptionHandling.SuppressedExceptionDemo.addSuppressedException(SuppressedExceptionDemo.java:15)
        ... 1 more
*/

In case of `try-with-resources`, exception can be thrown:
- while instantiating the resource inside try-with statement. In this case try block will not be executed and exception thrown by try-with statement will be thrown.
- if instantiation of resource is successful, and try block throws an exception and exception is also thrown while closing the resource, then the exception thrown while closing resource is suppressed by the exception thrown from try block. Java automatically adds the suppressed exception.
- if we provide explicit finally block and exception is thrown from that block it will suppress all other exception. (This explicit finally block executes after resources are closed)

In [ ]:
// No suppressed exceptions
public static void tryWithStatementException() throws Exception {
    try(InputStream in = new FileInputStream("abc")) {
        throw new Exception("Exception thrown from try block");
    }
}

public static void main(String[] args) throws Exception {
    tryWithStatementException();
}

/* Error log 
Exception in thread "main" java.io.FileNotFoundException: abc (The system cannot find the file specified)
    at java.base/java.io.FileInputStream.open0(Native Method)
    at java.base/java.io.FileInputStream.open(FileInputStream.java:213)
    at java.base/java.io.FileInputStream.<init>(FileInputStream.java:152)
    at java.base/java.io.FileInputStream.<init>(FileInputStream.java:106)
    at exceptionHandling.SuppressedExceptionDemo.tryWithStatementException(SuppressedExceptionDemo.java:37)
    at exceptionHandling.SuppressedExceptionDemo.main(SuppressedExceptionDemo.java:45)
*/

In [ ]:
// Exception while closing resource
public static void exceptionWhileClosingResource() throws Exception {
    try(SomeResource resource = new SomeResource()) {
        throw new Exception("Exception thrown from try block");
    }
}

public static void main(String[] args) throws Exception {
    exceptionWhileClosingResource();
}

class SomeResource implements AutoCloseable {
    @Override
    public void close() throws Exception {
        throw new Exception("Exception thrown while closing resource");
    }
}

/* Error log contains suppressed method. Java automatically added.
Exception in thread "main" java.lang.Exception: Exception thrown from try block
    at exceptionHandling.SuppressedExceptionDemo.exceptionWhileClosingResource(SuppressedExceptionDemo.java:44)
    at exceptionHandling.SuppressedExceptionDemo.main(SuppressedExceptionDemo.java:51)
    Suppressed: java.lang.Exception: Exception thrown while closing resource
        at exceptionHandling.SomeResource.close(SuppressedExceptionDemo.java:58)
        at exceptionHandling.SuppressedExceptionDemo.exceptionWhileClosingResource(SuppressedExceptionDemo.java:43)
        ... 1 more
*/

In [ ]:
// Resource close related exception suppressed in finally
public static void exceptionWhileClosingResourceSuppressed() throws Exception {
    try(SomeResource resource = new SomeResource()) {
        throw new Exception("Exception thrown from try block");
    } finally {
        throw new Exception("Exception thrown from finally block");
    }
}

public static void main(String[] args) throws Exception {
    exceptionWhileClosingResourceSuppressed();
}

/* Error log
Exception in thread "main" java.lang.Exception: Exception thrown from finally block
    at exceptionHandling.SuppressedExceptionDemo.exceptionWhileClosingResourceSuppressed(SuppressedExceptionDemo.java:52)
    at exceptionHandling.SuppressedExceptionDemo.main(SuppressedExceptionDemo.java:59)
*/

## Custom Exceptions
To create a custom checked exception, extend `Exception` class. To create a custom unchecked exception, extend `RuntimeException`. The `Exception` class contains the following helpful methods: 

**getMessage():** returns string value representation of what caused the exception to occur:

In [4]:
try {
    int i = 5 / 0;
} catch (Exception e) {
    System.err.println(e.getMessage());
}

/ by zero


**getStackTrace():** provides information about what methods were called in leading upto the exception:

In [5]:
try {
    int i = 5 / 0;
} catch (Exception e) {
    StackTraceElement[] elements = e.getStackTrace();
    for (StackTraceElement element : elements) {
        System.err.println(element.getFileName()
                + ":" + element.getLineNumber()
                + ">> "
                + element.getMethodName() + "()");
    }
}

// Below is the stacktrace information in JShell

$JShell$16.java:16>> do_it$()
DirectMethodHandleAccessor.java:103>> invoke()
Method.java:580>> invoke()
IJavaExecutionControl.java:95>> lambda$execute$1()
FutureTask.java:317>> run()
ThreadPoolExecutor.java:1144>> runWorker()
ThreadPoolExecutor.java:642>> run()
Thread.java:1583>> run()


**getCause():** sometimes we re-throw an exception in another more specific exception class like: 

In [6]:
// Define the below four constructor variants
class MyException extends Exception {
    public MyException () {

    }

    public MyException (String message) {
        super (message);
    }

    // This variant lets us define the cause of this exception
    // This is the correct variant to use when we re-throw exception, it will retain stacktrace
    public MyException (Throwable cause) {
        super (cause);
    }

    // This variant lets us define the cause of this exception along with message
    // This is the correct variant to use when we re-throw exception, it will retain stacktrace
    public MyException (String message, Throwable cause) {
        super (message, cause);
    }
}

MyException e = new MyException("My Custom Exception", new RuntimeException("An error occured"));
System.err.println(e.getCause().getMessage());

An error occured


This is also referred to as *chaining exceptions* and should be used in scenarios when the thrown exception was caused by another exception.

In [ ]:
try {
    Response response = httpClient.get(URL);
} catch (IOException e) {
    throw new ConnectionException("Unable to connect to " + URL, e);
}

/* Error log would look like:
Exception in thread "main" com.example.ConnectionException: Unable to connect to https://google.com
    ....
    ....
Caused by java.lang.IOException
    ....
*/

**printStackTrace():** defined in `Throwable`, it roughly works like this:
1. Print the message
```java
printStream.println(this);
```
2. Print the `StackElement`s
```java
for (StackTraceElement traceElement : trace)
    printStream.println("\tat " + traceElement);
```
3. Print suppressed exceptions
```java
for (Throwable se : getSuppressed())
    se.printStackTrace();
```
4. Print cause
```java
Throwable cause = getCause();
cause.printStackTrace();
```

## Assertions
Assertions are generally used during development. All assertions are ignored if the program is launched normally. In order to consider assertions we need to pass `-ea` VM arguments.  

The `assert` keyword is used to define assertions

In [ ]:
public getCount(int c){
    assert (c >= 0);
    // ...
}

In order to describe the assertion we can pass a string (or any object)

In [ ]:
public getCount(int c){
    assert (c >= 0): "Count must be greater than or equal to zero";
    // ...
}